In [1]:
import numpy as np
import pandas as pd

In [2]:
from bs4 import BeautifulSoup

In [3]:
import requests

In [4]:
url = 'https://www.chinatimes.com/search/擎天崗?chdtv&page'

In [15]:
res = requests.get(url)
# print(res.text) #查看抓到的原始碼的長相 (因為很長，這邊就不列出來了)

In [6]:
soup = BeautifulSoup(res.text, 'lxml')

In [7]:
count = soup.find('span',{'class':'search-result-count'}).get_text() #看有幾筆資料
print(count)

25


In [8]:
df = pd.DataFrame({'title':[0] ,'context': [0],'url':[0] }) #創一個表格
print(df)

   title  context  url
0      0        0    0


In [9]:
for i in range(0,int(np.ceil(25/20))): #np.ceil是無條件進位，先判斷頁數有幾頁，是否需要換頁
    url = 'https://www.chinatimes.com/search/擎天崗?chdtv&page='+str(i+1)
    res = requests.get(url) #取得網頁資料
    soup = BeautifulSoup(res.text, 'lxml') #將網頁資料存到BeautifulSoup4可以讀取的格式
    for context in soup.find_all('div',{'class':'col'}): #從網頁的程式碼中找需要的資料。
        df1 = pd.DataFrame({'title':[context.find('h3',{'class':'title'}).get_text()],
                            'context':[context.find('p',{'class':'intro'}).get_text()],
                            'url':[context.find('a').get('href')]})
        df = pd.concat([df,df1],axis=0)

In [10]:
df.reset_index(inplace=True) #重設index

In [12]:
df.drop([0],axis=0,inplace=True) #把不要的資料欄位移除
df.drop(['index'],axis=1,inplace=True)

In [13]:
df

,title,context,url
1,台版撒哈拉沙漠！抓準日落賞沙洲最美,\r\n\r\n\r\n▍頂頭額沙洲\r\n\r\n被封為「台版撒哈拉沙漠」、「台版絲路」的...,https://www.chinatimes.com/fashion/20190415003...
2,擎天崗牛隻襲擊遊客 美籍男子等2人受傷,11日下午3時許，一群遊客在擎天崗草原漫步時，在附近放養的野生水牛疑似受驚，朝人群衝撞，混亂...,https://www.chinatimes.com/realtimenews/201808...
3,擎天崗突起大霧 7外籍生伸手不見五指報警求救,\r\n7名外籍男女學生日前相約赴陽明山區擎天崗爬山，卻因錯過末班公車以及山區大霧籠罩迷路，...,https://www.chinatimes.com/realtimenews/201806...
4,擎天崗野餐趣 大稻埕賞日落,遼闊的擎天崗，據說早在1934年，日本人在此種植了地毯草，當時稱為「大嶺牧場」，至台灣光復...,https://www.chinatimes.com/newspapers/20180317...
5,不只擎天崗！陽明山打卡秘境TOP3「超夢幻黑森林必訪」,想到陽明山，通常我們第一個想到的都是文化大學後山夜景、擎天崗草原，但真的只有這樣嗎？其實陽明...,https://www.chinatimes.com/realtimenews/201803...
6,陽明山擎天崗野牛亂來 扯出李登輝取精之秘,陽明山擎天崗草原風景優美，向來是北部民眾喜歡的熱門景點，但近來屢傳攻擊遊客事件，讓陽明山國家...,https://www.chinatimes.com/realtimenews/201802...
7,登擎天崗天黑濃霧受困 視訊119App報案獲救,台北市消防局25日晚間6時58分接獲警察局轉報，有2位民眾（含1位外籍人士）於士林區擎天崗環...,https://www.chinatimes.com/realtimenews/201801...
8,笑翻！想閃積水走路邊 下一秒糗「躺」入水,雨下不停！這幾天受到卡努颱風與東北風的共伴效應影響，全台各地降下驚人豪雨量，不少地方都紛紛傳...,https://www.chinatimes.com/hottopic/2017101500...
9,出動61警消搜救24小時 4名登山客早已返家睡覺,昨天下午2點多，台北市消防局獲報有4名登山客在擎天崗附近失蹤，立即通知志工、警消到場搜救，經...,https://www.chinatimes.com/realtimenews/201707...
10,警消迷霧救援 迷途登山情侶獲救,31歲謝姓及27歲馮姓一對情侶，昨天下午結伴前往陽明山區爬山，因不熟路況，在擎天崗荷蘭古道迷...,https://www.chinatimes.com/realtimenews/201603...


In [14]:
df.to_excel('中時電子報擎天崗爬蟲結果.xlsx')